# Make a table of the first and last year of data available for tas and pr for each CMIP6 model/member/scenario combination

In [1]:
import pandas as pd
import numpy as np
import fsspec
import xarray as xr
import os.path
from file_control import model_table_dir
from file_control import gmst_table_dir
from IPython.display import clear_output

In [2]:
# set the output file name
outfile = os.path.join(model_table_dir, 'all_zarr_models_first_last_year.csv')

In [3]:
# Use the GMST table to get a table of each model/member/scenario combination 
# available from Google Cloud
gmst = pd.read_csv(os.path.join(gmst_table_dir, 'CMIP6_GMST_table_all.csv'))
mms = gmst[['model','member','scenario']].drop_duplicates().copy()
mms

,model,member,scenario
0,GFDL-CM4,r1i1p1f1,historical
165,GFDL-CM4,r1i1p1f1,ssp245
251,GFDL-CM4,r1i1p1f1,ssp585
337,GFDL-ESM4,r1i1p1f1,historical
502,GFDL-ESM4,r1i1p1f1,ssp119
...,...,...,...
175553,ACCESS-CM2,r5i1p1f1,historical
175718,ACCESS-CM2,r5i1p1f1,ssp126
175804,ACCESS-CM2,r5i1p1f1,ssp245
175890,ACCESS-CM2,r5i1p1f1,ssp370


In [4]:
# Set up an output dataframe
outdf = mms.copy()
outdf['tas_first_year'] = np.nan
outdf['tas_last_year'] = np.nan
outdf['pr_first_year'] = np.nan
outdf['pr_last_year'] = np.nan
outdf

,model,member,scenario,tas_first_year,tas_last_year,pr_first_year,pr_last_year
0,GFDL-CM4,r1i1p1f1,historical,NaN,NaN,NaN,NaN
165,GFDL-CM4,r1i1p1f1,ssp245,NaN,NaN,NaN,NaN
251,GFDL-CM4,r1i1p1f1,ssp585,NaN,NaN,NaN,NaN
337,GFDL-ESM4,r1i1p1f1,historical,NaN,NaN,NaN,NaN
502,GFDL-ESM4,r1i1p1f1,ssp119,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
175553,ACCESS-CM2,r5i1p1f1,historical,NaN,NaN,NaN,NaN
175718,ACCESS-CM2,r5i1p1f1,ssp126,NaN,NaN,NaN,NaN
175804,ACCESS-CM2,r5i1p1f1,ssp245,NaN,NaN,NaN,NaN
175890,ACCESS-CM2,r5i1p1f1,ssp370,NaN,NaN,NaN,NaN


In [5]:
# for each model/member/scenario combination, for tas and pr, identify the first
# and last year of data available
df = pd.read_csv('https://storage.googleapis.com/cmip6/' + 
                 'cmip6-zarr-consolidated-stores.csv')
varnms = ['tas','pr']

for i in range(mms.shape[0]):
    print(i, end='\r')
    for varnm in varnms:
        # grab the data
        df_zarr = df.query("table_id == 'Amon' and " + 
                           "variable_id == '" + varnm + "' and " + 
                           "experiment_id == '" + mms.iloc[i]['scenario'] + 
                           "' and source_id == '" + mms.iloc[i]['model'] + 
                           "' and member_id == '" + mms.iloc[i]['member'] + "'")
        
        if df_zarr.shape[0] == 0: 
            # if the data is not available for this 
            # model/member/scenario/variable
            outdf.loc[outdf.index[i], varnm + '_first_year'] = np.nan
            outdf.loc[outdf.index[i], varnm + '_last_year'] = np.nan
        else:
            zstore = df_zarr.zstore.values[0]
            # create a mutable-mapping-style interface to the store
            mapper = fsspec.get_mapper(zstore)
            # open it using xarray and zarr
            zz = xr.open_zarr(mapper, consolidated=True)
    
            # assign the first and last years to the output dataframe
            outdf.loc[outdf.index[i], varnm + '_first_year'] = zz[
                "time.year"].min().values
            outdf.loc[outdf.index[i], varnm + '_last_year'] = zz[
                "time.year"].max().values

clear_output()

In [6]:
# How common is it for a future scenario to not be available for the full 
# 2015-2099 period?

# check tas 
# for ssp534-over, quite a few models don't start until 2040. Maybe the earlier 
# part of ssp534-over can be sourced from another scenario?
outdf.loc[(outdf['scenario'] != 'historical') & 
          (outdf['tas_first_year'] > 2015)]
# quite a few cases (for ssp370 and ssp245) where data doesn't go to 2099
outdf.loc[(outdf['scenario'] != 'historical') & 
          (outdf['tas_last_year'] < 2099)]

# check pr
outdf.loc[(outdf['scenario'] != 'historical') & 
          (outdf['pr_first_year'] > 2015)]
outdf.loc[(outdf['scenario'] != 'historical') & 
          (outdf['pr_last_year'] < 2099)]



,model,member,scenario,tas_first_year,tas_last_year,pr_first_year,pr_last_year


In [7]:
# save the table

outdf.to_csv(outfile, index=False)

In [8]:
outdf = pd.read_csv(outfile)
outdf.loc[(outdf['scenario'] != 'historical') & 
          (outdf['tas_first_year'] > 2015)]


,model,member,scenario,tas_first_year,tas_last_year,pr_first_year,pr_last_year
36,IPSL-CM6A-LR,r1i1p1f1,ssp534-over,2040.0,2300.0,2040.0,2300.0
74,MRI-ESM2-0,r1i1p1f1,ssp534-over,2040.0,2300.0,2040.0,2300.0
230,CanESM5,r1i1p1f1,ssp534-over,2040.0,2300.0,2040.0,2300.0
257,CanESM5,r5i1p1f1,ssp534-over,2040.0,2100.0,2040.0,2100.0
266,CanESM5,r4i1p1f1,ssp534-over,2040.0,2100.0,2040.0,2100.0
299,CanESM5,r3i1p1f1,ssp534-over,2040.0,2100.0,2040.0,2100.0
320,CanESM5,r2i1p1f1,ssp534-over,2040.0,2100.0,2040.0,2100.0
487,UKESM1-0-LL,r1i1p1f2,ssp534-over,2040.0,2100.0,2040.0,2100.0
495,UKESM1-0-LL,r3i1p1f2,ssp534-over,2040.0,2100.0,2040.0,2100.0
503,UKESM1-0-LL,r2i1p1f2,ssp534-over,2040.0,2100.0,2040.0,2100.0
